In [1]:
# importando a biblioteca gurobi

import gurobipy as gp
from gurobipy import GRB


In [2]:
# inserindo os parâmetros do modelo

# demanda dos armazens (em milhares)
demanda = [15, 18, 14, 20]

# capacidade das fábricas (em milhares)
capacidade  = [20, 22, 17, 19, 18]

# custo fixo de cada fábrica
custo_fixo = [12000, 15000, 17000, 13000, 16000]

# custo de transporte por milhares de unidades
custo_transporte = [[4000, 2000, 3000, 2500, 4500],
                    [2500, 2600, 3400, 3000, 4000],
                    [1200, 1800, 2600, 4100, 3000],
                    [2200, 2600, 3100, 3700, 3200]]

#estruturando os dados de fábricas e armazens
fabricas = range(len(capacidade))
armazens = range(len(demanda))


In [20]:
# modelagem do problema
m = gp.Model('facilidades')

# adicionado variável decisória binária fábrica aberta=1, fechada=0
open = m.addVars(fabricas,
                vtype=GRB.BINARY,
                obj=custo_fixo,
                name='open')

# adicionado variável decisória contínua para definir o fluxo ideal de unidades para transporte
transporte = m.addVars(armazens, fabricas, obj=custo_transporte, name='transporte')

# agora definimos o objetivo do modelo, que será minimizar o custo_fixo e o custo_transporte
m.modelsense = GRB.MINIMIZE

# adicionando as restrições de capacidade de produção
m.addConstrs(
    (transporte.sum('*', f) <= capacidade[f]*open[f] for f in fabricas), 'capacidade')

#adicionado as restrições de demanda
m.addConstrs(
    (transporte.sum(a) == demanda[a] for a in armazens), 'demanda')

# salvando o modelo
m.write("facilidades.lp")


# agora podemos abrir todas as fábricas para iniciar o modelo
for f in fabricas:
    open[f].Start = 1.0

# agora iremos fechar a fabrica que apresenta o maior custo_fixo
# similarmente poderiamos abrir/fechar a fabrica que apresenta menor custo_fixo
print('Iniciando:')
fixo_maximo = max(custo_fixo)
for f in fabricas:
    if custo_fixo[f] == fixo_maximo:
        open[f].Start = 0.0
        print('Fechando Fabrica %s' % f)
        break
print('')

# solucinando o modelo
m.optimize()

# imprimindo a solução
print('\nCusto total: R$%g' % m.ObjVal)
print('Solução:')
for f in fabricas:
    if open[f].x > 0.99:
        print('\nFabrica %s aberta!' % f)
        for a in armazens:
            if transporte[a, f].x > 0:
                print('Transportando %g unidades para o armazem %s' %
                      (transporte[a, f].x, a))
    else:
        print('\nFabrica %s fechada!' % f)


Iniciando:
Fechando Fabrica 2

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 9 rows, 25 columns and 45 nonzeros
Model fingerprint: 0x36b45dc0
Variable types: 20 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]

User MIP start produced solution with objective 210500 (0.01s)
Loaded user MIP start with objective 210500

Presolve time: 0.00s
Presolved: 9 rows, 25 columns, 45 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)

Root relaxation: objective 1.998333e+05, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 199833.333    0    1 210500.000 199833.333  5.07%     -    0s